In [1]:
import pandas as pd
import numpy as np
import os
import warnings
from scipy.sparse import csr_matrix
warnings.filterwarnings("ignore")
os.chdir('E:\\movieLens dataset')

In [2]:
# rating = pd.read_csv('u.rating', sep='\t', names=['userId', 'movieId', 'rating', 'timestamp'])

In [3]:
data = pd.read_csv('u.data', sep='\t', names=['userId', 'movieId', 'rating', 'timestamp'])
movie = pd.read_csv('u.item', sep='\|', header=None)
movie = movie[[0, 1, 2]]
movie.columns = ['movieId', 'title', 'date']

In [4]:
temp = data.groupby('movieId').count()['rating'].sort_values(ascending=False).to_frame()
temp1 = data.groupby('userId').count()['rating'].sort_values(ascending=False).to_frame()
less3 = temp[temp['rating']<3]
data = data[~data['movieId'].isin(less3.index)]
for i in [2,4,6,8,10,12,14,16,18,20]:
    data.drop(data[(data['userId']==1) & (data['movieId']==i)].index, inplace=True)
pivot = pd.pivot_table(data=data, index='userId', columns='movieId', values='rating')
pivot['userId'] = pivot.index

In [5]:
pivot = pivot.drop(columns='userId')

pivot

movieId,1,2,3,4,5,6,7,8,9,10,...,1615,1620,1622,1623,1628,1639,1643,1652,1658,1664
userId,,,,,,,,,,,,,,,,,,,,,
1,5.0,NaN,4.0,NaN,3.0,NaN,4.0,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,2.0,NaN,NaN,4.0,5.0,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,5.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
R_demeaned = pivot.values

In [7]:
users_mean = np.array(pivot.mean(axis=1))
R_demeaned = pivot.sub(pivot.mean(axis=1), axis=0)
R_demeaned = R_demeaned.fillna(0).values
R_demeaned_sparse = csr_matrix(R_demeaned)


In [8]:
R_demeaned.shape

(943, 1473)

In [30]:
R

NameError: name 'R' is not defined

In [25]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(R_demeaned_sparse, k = 100)

In [26]:
sigma = np.diag(sigma)

In [27]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + users_mean.reshape(-1, 1)
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = pivot.columns)

In [28]:
all_user_predicted_ratings.shape

(943, 1473)

In [29]:
preds_df

movieId,1,2,3,4,5,6,7,8,9,10,...,1615,1620,1622,1623,1628,1639,1643,1652,1658,1664
0,5.169464,3.577566,4.080657,3.858493,3.128428,3.515796,3.986368,3.397134,5.336970,3.636398,...,3.558845,3.520197,3.540155,3.599011,3.661246,3.578943,3.603311,3.589450,3.592851,3.655801
1,3.893762,3.664815,3.535752,3.671312,3.727614,3.776687,3.709945,3.702302,3.737384,3.547434,...,3.713788,3.743520,3.715077,3.709162,3.733719,3.720375,3.711716,3.696557,3.706165,3.696345
2,3.003072,2.746606,2.809504,2.653619,2.635395,2.782282,2.860441,2.834141,3.120515,2.626002,...,2.780596,2.759271,2.772296,2.806019,2.804793,2.786155,2.822258,2.814296,2.806878,2.811732
3,4.349169,4.328205,4.358445,4.368180,4.313854,4.350057,4.367533,4.357516,4.172436,4.362829,...,4.330728,4.287039,4.313582,4.334195,4.329403,4.330676,4.334562,4.351712,4.342482,4.313913
4,3.718374,2.901447,2.530371,3.178099,2.872111,2.849089,2.994325,2.816373,3.265005,2.670597,...,2.815088,2.839419,2.905577,2.868014,2.844083,2.822830,2.856366,2.847430,2.843865,2.854137
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,4.388595,4.276815,4.051897,4.412623,4.250660,4.228652,4.337937,4.075864,4.777883,4.260009,...,4.216253,4.139712,4.269514,4.269368,4.263532,4.262151,4.238689,4.239157,4.248769,4.275777
939,3.721750,3.474694,3.415876,2.820139,3.518335,3.507861,3.762915,3.963528,2.844977,3.464152,...,3.490663,3.438260,3.446106,3.466495,3.440455,3.487999,3.466243,3.446314,3.462107,3.468734
940,4.495650,3.986761,4.094431,4.008598,4.114924,4.100884,4.065825,4.137521,3.992469,4.106605,...,4.044395,3.995718,4.046871,4.035612,4.052479,4.027700,4.028579,4.043773,4.048009,4.035440
941,4.273735,4.263225,4.395606,4.335236,4.272257,4.312117,4.051761,4.442265,4.365152,4.283016,...,4.273937,4.194727,4.283314,4.269005,4.276829,4.267482,4.274458,4.299739,4.284621,4.261690


In [21]:
preds_sps = preds_df.values
pivot_sps = pivot.values
#Для подсчета rmse были удалены NaN из изначального ДФ и элементы с такими же индексами из предсказаний
pivot_sps_1 = pivot_sps[~np.isnan(pivot_sps)]
preds_sps_1 = preds_sps[~np.isnan(pivot_sps)]

print(sqrt(mse([3, 3, 5, 1, 3], [3.577566, 3.858493, 3.515796, 3.397134, 3.636398])))

1.3729317577637281


In [16]:
from sklearn.metrics import mean_squared_error as mse
from math import sqrt
err = mse(pivot_sps_1, preds_sps_1)
rmse = sqrt(err)
print(rmse)

1.0079154370856362


In [53]:
def recommend_movies(predictions_df, userID, movies_df, original_ratings_df, num_recommendations=5):
    
    # Get and sort the user's predictions
    user_row_number = userID - 1 # UserID starts at 1, not 0
    sorted_user_predictions = predictions_df.iloc[user_row_number].sort_values(ascending=False)
    
    # Get the user's data and merge in the movie information.
    user_data = original_ratings_df[original_ratings_df.userId == (userID)]
    user_full = (user_data.merge(movies_df, how = 'left', left_on = 'movieId', right_on = 'movieId').
                     sort_values(['rating'], ascending=False)
                 )

    print ('User {0} has already rated {1} movies.'.format(userID, user_full.shape[0]))
    print ('Recommending the highest {0} predicted ratings movies not already rated.'.format(num_recommendations))
    
    # Recommend the highest predicted rating movies that the user hasn't seen yet.
    recommendations = (movies_df[~movies_df['movieId'].isin(user_full['movieId'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'movieId',
               right_on = 'movieId').
         rename(columns = {user_row_number: 'predictions'}).
         sort_values('predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )

    return user_full, recommendations

already_rated, predictions = recommend_movies(preds_df, 837, movie, data, 10)

User 837 has already rated 46 movies.
Recommending the highest 10 predicted ratings movies not already rated.


In [54]:
predictions

,movieId,title,date
654,690,Seven Years in Tibet (1997),01-Jan-1997
377,408,"Close Shave, A (1995)",28-Apr-1996
184,196,Dead Poets Society (1989),01-Jan-1989
646,682,I Know What You Did Last Summer (1997),17-Oct-1997
271,301,In & Out (1997),19-Sep-1997
116,126,"Spitfire Grill, The (1996)",06-Sep-1996
107,116,Cold Comfort Farm (1995),23-Apr-1996
188,200,"Shining, The (1980)",01-Jan-1980
122,132,"Wizard of Oz, The (1939)",01-Jan-1939
142,153,"Fish Called Wanda, A (1988)",01-Jan-1988


In [55]:
already_rated.head(10)

,userId,movieId,rating,timestamp,title,date
44,837,740,5,875722123,Jane Eyre (1996),05-Apr-1996
37,837,1009,5,875721765,Stealing Beauty (1996),14-Jun-1996
9,837,283,5,875722069,Emma (1996),02-Aug-1996
36,837,125,5,875722032,Phenomenon (1996),29-Jun-1996
11,837,289,5,875721539,Evita (1996),25-Dec-1996
24,837,151,5,875721734,Willy Wonka and the Chocolate Factory (1971),01-Jan-1971
23,837,258,4,875721473,Contact (1997),11-Jul-1997
14,837,20,4,875721919,Angels and Insects (1995),01-Jan-1995
22,837,294,4,875721502,Liar Liar (1997),21-Mar-1997
27,837,328,4,875721604,Conspiracy Theory (1997),08-Aug-1997


In [33]:
data[data['userId']==837]

,userId,movieId,rating,timestamp
56961,837,476,3,875722225
57098,837,845,4,875722392
57321,837,220,4,875722007
57745,837,225,3,875722371
57771,837,274,4,875721989
58066,837,535,1,875722246
58715,837,19,4,875721948
59369,837,628,3,875722225
59875,837,596,3,875721969
60491,837,283,5,875722069
